# BB84 — Bob Role



In [3]:
import qiskit

In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()


qiskit_runtime_service.__init__:WARNING:2025-11-15 08:11:21,721: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

# Load IBM account
service = QiskitRuntimeService()

# Select backend (hardware)
backend = service.backend("ibm_fez")

# Make circuit
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

# ⭐ Transpile for target hardware ⭐
qc_t = transpile(qc, backend)

# Run on hardware through SamplerV2
sampler = SamplerV2(backend)
job = sampler.run([qc_t])   # ONLY transpiled circuits allowed
result = job.result()

print(result)


qiskit_runtime_service.__init__:WARNING:2025-11-15 08:11:52,397: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-15 08:11:52,400: Using instance: open-instance, plan: open


PrimitiveResult([SamplerPubResult(data=DataBin(c=BitArray(<shape=(), num_shots=4096, num_bits=1>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-11-15 02:43:11', stop='2025-11-15 02:43:18', size=4096>)])}, 'version': 2})


In [7]:
# bob_qiskit.py
from flask import Flask, request, jsonify
import numpy as np, time, requests, random
try:
    from qiskit import QuantumCircuit, transpile
    from qiskit.primitives import Sampler
    _HAS_SAMPLER = True
except Exception:
    from qiskit import QuantumCircuit
    _HAS_SAMPLER = False

app = Flask(__name__)

ALICE_IP = "127.0.0.1"
ALICE_SET_KEY_ENDPOINT = f"http://{ALICE_IP}:5000/set_final_key"

N = 0
bob_bases = []
received_preps = []
bob_measurements = {}
final_key = None

if _HAS_SAMPLER:
    sampler = Sampler()

def build_measure_circuit(bit, prep_basis, bob_basis):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)
    if prep_basis == 1:
        qc.h(0)
    if bob_basis == 1:
        qc.h(0)
    qc.measure(0, 0)
    return qc

def analytic_measure(bit, prep_basis, bob_basis):
    # follows BB84 single-qubit physics exactly
    if prep_basis == bob_basis:
        return int(bit)
    else:
        return int(random.getrandbits(1))

def measure_with_sampler(bit, prep_basis, bob_basis):
    qc = build_measure_circuit(bit, prep_basis, bob_basis)
    # run sampler
    job = sampler.run([qc])
    result = job.result()
    try:
        counts = result.get_counts(0)
        measured = int(list(counts.keys())[0])
    except Exception:
        try:
            measured = int(list(result.quasi_dists[0].to_dict().keys())[0])
        except Exception:
            measured = int(random.getrandbits(1))
    return measured

def measure_on_backend(bit, prep_basis, bob_basis):
    if _HAS_SAMPLER:
        return measure_with_sampler(bit, prep_basis, bob_basis)
    else:
        return analytic_measure(bit, prep_basis, bob_basis)

@app.route("/receive_from_eve", methods=["POST"])
def receive_from_eve():
    global received_preps, bob_bases, N, bob_measurements
    data = request.json
    received_preps = data.get("preps", [])
    N = len(received_preps)
    bob_bases = np.random.randint(2, size=N).tolist()
    outcomes = []
    for i, item in enumerate(received_preps):
        bit = int(item["bit"])
        prep_basis = int(item["basis"])
        measured = measure_on_backend(bit, prep_basis, bob_bases[i])
        outcomes.append(int(measured))
    bob_measurements = {}
    for i, item in enumerate(received_preps):
        idx = item["idx"]
        bob_measurements[idx] = int(outcomes[i])
    globals()['bob_bases'] = bob_bases
    globals()['received_preps'] = received_preps
    globals()['bob_measurements'] = bob_measurements
    print(f"Bob: measured {N} qubits (sampler available: { _HAS_SAMPLER })")
    return jsonify({"status": "measured", "n": N})

@app.route("/receive_alice_bases", methods=["POST"])
def receive_alice_bases():
    global final_key
    alice_bases = request.json.get("bases", [])
    if len(alice_bases) == 0:
        return jsonify({"error": "no bases provided"}), 400
    indices = [item["idx"] for item in received_preps]
    a_bases_aligned = [int(alice_bases[i]) for i in range(len(indices))]
    b_bases_aligned = [int(bob_bases[i]) for i in range(len(indices))]
    alice_bits_aligned = [int(item.get("bit")) for item in received_preps]
    bob_bits_aligned = [bob_measurements[item["idx"]] for item in received_preps]
    sift_mask = [1 if a_bases_aligned[i] == b_bases_aligned[i] else 0 for i in range(len(indices))]
    sifted_alice = [alice_bits_aligned[i] for i in range(len(indices)) if sift_mask[i]]
    sifted_bob = [bob_bits_aligned[i] for i in range(len(indices)) if sift_mask[i]]
    if len(sifted_alice) == 0:
        qber = None
    else:
        errors = sum(1 for i in range(len(sifted_alice)) if sifted_alice[i] != sifted_bob[i])
        qber = errors / len(sifted_alice)
    final_key = sifted_bob[:]
    print("Bob: Final key ready (len={}): {}".format(len(final_key), final_key))
    # automatically push to Alice
    try:
        resp = requests.post(ALICE_SET_KEY_ENDPOINT, json={"sifted_bob": final_key}, timeout=20)
        print("Bob -> Alice set_key status:", resp.status_code)
    except Exception as e:
        print("Bob -> Alice set_key failed:", e)
    return jsonify({
        "sifted_len": len(sifted_alice),
        "qber": qber,
        "sifted_alice": sifted_alice,
        "sifted_bob": sifted_bob
    })

@app.route("/receive_ciphertext", methods=["POST"])
def receive_ciphertext():
    global final_key
    data = request.json
    ciphertext = bytes(data.get("ciphertext", []))
    print("\n======== Bob RECEIVED CIPHERTEXT ========")
    print("Ciphertext bytes:", list(ciphertext))
    if final_key is None or len(final_key) == 0:
        print("ERROR: Bob has no key yet!")
        return jsonify({"error": "Bob has no key yet"}), 400
    kb = list(final_key)
    if len(kb) < 8:
        kb = kb + [0]*(8 - len(kb))
    key_bytes = bytes([int("".join(str(b) for b in kb[i:i+8]), 2) for i in range(0, len(kb), 8)])
    repeated_key = (key_bytes * ((len(ciphertext) // len(key_bytes)) + 1))[:len(ciphertext)]
    plaintext_bytes = bytes([c ^ k for c, k in zip(ciphertext, repeated_key)])
    try:
        plaintext = plaintext_bytes.decode("utf-8")
    except Exception:
        plaintext = str(plaintext_bytes)
    print("Decrypted message:", plaintext)
    print("=========================================\n")
    return jsonify({"ciphertext_bytes": list(ciphertext), "decrypted_message": plaintext})

if __name__ == "__main__":
    print("Bob (QKD sim) server running on port 5002  (sampler available: {})".format(_HAS_SAMPLER))
    app.run(host="0.0.0.0", port=5002)


Bob (QKD sim) server running on port 5002  (sampler available: False)
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://192.168.0.107:5002
Press CTRL+C to quit
